In [ ]:
# check whether to use a gpu or cpu
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Simple examples

In [ ]:
# make a forward pass through the model
from transformers import BioGptConfig
from histogpt.models import HistoGPTForCausalLM, PerceiverResamplerConfig

histogpt = HistoGPTForCausalLM(BioGptConfig(), PerceiverResamplerConfig())
histogpt = histogpt.to(device)

text = torch.randint(0, 42384, (1, 256)).to(device)
image = torch.rand(1, 1024, 768).to(device)

print(histogpt(text, image).logits.size())

In [ ]:
# generate text autoregressively
from histogpt.helpers.inference import generate

output = generate(
    model=histogpt,
    prompt=torch.randint(0, 42384, (1, 2)),
    image=torch.rand(1, 2, 768),
    length=256,
    top_k=40,
    top_p=0.95,
    temp=0.7,
    device=device
)

print(output.size())

# Generate reports from features

In [ ]:
# load model weights
PATH = 'histogpt-1b-6k-pruned.pthe'
state_dict = torch.load(PATH, map_location=device)
histogpt.load_state_dict(state_dict, strict=True)

In [ ]:
# get text prompt and image features
import h5py
from transformers import BioGptTokenizer

tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

prompt = 'Final diagnosis:'
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

with h5py.File('2023-03-06 23.51.44.h5e', 'r') as f:
    features = f['feats'][:]
    features = torch.tensor(features).unsqueeze(0).to(device)

In [ ]:
# generate the pathology report
output = generate(
    model=histogpt,
    prompt=prompt,
    image=features,
    length=256,
    top_k=40,
    top_p=0.95,
    temp=0.7,
    device=device
)

decoded = tokenizer.decode(output[0, 1:])
print(decoded)

# Generate reports from images

In [ ]:
# start patching and extracting features
import os
import shutil
from histogpt.helpers.patching import main, PatchingConfigs

try:
  os.mkdir('slide_folder')
  os.mkdir('save_folder')
  shutil.move("2023-03-06 23.51.44.ndpie", "slide_folder/2023-03-06 23.51.44.ndpi")
except Exception:
  pass

configs = PatchingConfigs()
configs.slide_path = 'slide_folder'
configs.save_path = 'save_folder'
configs.model_path = 'ctranspath.pthe'
configs.patch_size = 256
configs.white_thresh = [170, 185, 175]
configs.edge_threshold = 2
configs.resolution_in_mpp = 0.0
configs.downscaling_factor = 4.0
configs.batch_size = 16

main(configs)

In [ ]:
# having the feature vectors, we can proceed as above
import h5py

with h5py.File('save_folder/h5_files/256px_ctranspath_0.0mpp_4.0xdown_normal/2023-03-06 23.51.44.h5', 'r') as f:
    features = f['feats'][:]
    print(features.shape)

In [ ]:
with h5py.File('2023-03-06 23.51.44.h5e', 'r') as f:
    features = f['feats'][:]
    print(features.shape)

# Load the largest, most powerful model